In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import os 


In [ ]:
train_dir='../input/plant-seedlings-classification/train'
test_dir='../input/plant-seedlings-classification/test'

categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']


In [ ]:
print(categories)

In [ ]:
def category_to_label(category):
    if category == 'Black-grass': return [1,0,0,0,0,0,0,0,0,0,0,0]
    elif category == 'Charlock': return [0,1,0,0,0,0,0,0,0,0,0,0]
    elif category == 'Cleavers': return [0,0,1,0,0,0,0,0,0,0,0,0]
    elif category == 'Common Chickweed': return [0,0,0,1,0,0,0,0,0,0,0,0]
    elif category == 'Common wheat': return [0,0,0,0,1,0,0,0,0,0,0,0]
    elif category == 'Fat Hen': return [0,0,0,0,0,1,0,0,0,0,0,0]
    elif category == 'Loose Silky-bent': return [0,0,0,0,0,0,1,0,0,0,0,0]
    elif category == 'Maize': return [0,0,0,0,0,0,0,1,0,0,0,0]
    elif category == 'Scentless Mayweed': return [0,0,0,0,0,0,0,0,1,0,0,0]
    elif category == 'Shepherds Purse': return [0,0,0,0,0,0,0,0,0,1,0,0]
    elif category == 'Small-flowered Cranesbill': return [0,0,0,0,0,0,0,0,0,0,1,0]
    elif category == 'Sugar beet': return [0,0,0,0,0,0,0,0,0,0,0,1] 

In [ ]:
import os 
import cv2
from random import shuffle
def create_train_data():
    train=[]
    for category in categories:
        for img in os.listdir(os.path.join(train_dir,category)):
            label=category_to_label(category)
            image_path=os.path.join(train_dir,category,img)
            img=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
            img=cv2.resize(img,(128,128))
            train.append([np.array(img),label])
    
    shuffle(train)
    return(train)

In [ ]:
train_data=create_train_data()

In [ ]:
train_data

In [ ]:
def create_test_data():
    test=[]
    for img in os.listdir(test_dir):
        img_num = img
        image_path=os.path.join(test_dir,img)
        img=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(128,128))
        test.append([np.array(img),img_num])
    shuffle(test)
    return(test)

In [ ]:
test_data=create_test_data()

In [ ]:
test_data

In [ ]:
x_train=np.array([i[0] for i in train_data]).reshape(-1,128,128,1)
y_train=[i[1] for i in train_data]

In [ ]:
x_train.shape

In [ ]:
y_train=np.vstack(y_train)

In [ ]:
y_train

In [ ]:
y_train.shape

In [ ]:
x_test=np.array([i[0]for i in test_data])
test_image_name=[i[1] for i in test_data]

In [ ]:
x_test

In [ ]:
test_image_name

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=0.15,random_state=42)

In [ ]:
print(x_train.shape)

In [ ]:
x_valid.shape

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical


In [ ]:
model=Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(128, 128, 1)))  
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Conv2D(64, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))  
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal')  )
model.add(MaxPooling2D(pool_size=(2, 2)))  

model.add(Flatten())  
model.add(Dense(12, activation='softmax'))  

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    width_shift_range=0.2,  # 3.1.1 左右にずらす
    height_shift_range=0.2,  # 3.1.2 上下にずらす
    horizontal_flip=True,  # 3.1.3 左右反転
    # 3.2.1 Global Contrast Normalization (GCN) (Falseに設定しているのでここでは使用していない)
    samplewise_center=False,
    samplewise_std_normalization=False,
    zca_whitening=False)  # 3.2.2 Zero-phase Component Analysis (ZCA) Whitening (Falseに設定しているのでここでは使用していない)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                    steps_per_epoch=x_train.shape[0] // 100, epochs=30, validation_data=(x_valid, y_valid),callbacks=[early_stopping])

[early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, verbose=1,
          validation_data=(x_valid, y_valid), callbacks=[early_stopping])

In [ ]:
x_test=x_test.reshape(-1,128,128,1)

In [ ]:
x_test.shape

In [ ]:
pre=model.predict(x_test)

In [ ]:
pre.shape

In [ ]:
pre=np.argmax(pre,axis=1)

In [ ]:
pre